In [14]:
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import pandas as pd
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_log_error as msle
import numpy as np 

In [2]:
df = pd.read_csv("training_dataset3.csv")

In [3]:
X = df[['Num_escuelas',
 'Num_hospitales',
 'Num_iglesias',
 'Num_zonas_tipicas',
  "Hora_llegada_1",                                          
  "Hora_llegada_2",
  "Hora_llegada_3",                                          
  "Hora_llegada_4",
  "Hora_llegada_5",                                          
  "Hora_llegada_6",
  "Hora_llegada_7",                                          
  "Hora_llegada_8",
  "Hora_llegada_9",                                          
  "Hora_llegada_10",
  "Hora_llegada_11",                                          
  "Hora_llegada_12",
  "Hora_llegada_13",                                          
  "Hora_llegada_14",
  "Hora_llegada_15",                                          
  "Hora_llegada_16",                                         
  "Hora_llegada_17",                                          
  "Hora_llegada_18",
  "Hora_llegada_19",                                          
  "Hora_llegada_20",
  "Hora_llegada_21",                                          
  "Hora_llegada_22",
  "Hora_llegada_23", ]]

y = df["Num_viajes"]

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42)

# XGBoost

Ajuste del modelo XGBoost

Para afinar los hiperparámetros de este modelo, recurrimos a utilizar una búsqueda en cuadrícula CV. Este enfoque nos permite probar diferentes valores de los hiperparámetros para encontrar los que ofrecen el mejor rendimiento

Más información: https://towardsdatascience.com/xgboost-fine-tune-and-optimize-your-model-23d996fab663

In [16]:
from sklearn.model_selection import GridSearchCV

params = { 'max_depth': [3,6],
           'learning_rate': [ 0.1, 0.5],
           'n_estimators': [100000],
           'colsample_bytree': [0.1, 0.01]}
xgbr = xgb.XGBRegressor(objective = "count:poisson",seed = 42)
clf = GridSearchCV(
                    estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_squared_log_error',
                   verbose=1)
clf.fit(X, y)
print("Best parameters:", clf.best_params_)
print("Lowest MSLE: ", (-clf.best_score_)**(1/2.0))

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best parameters: {'colsample_bytree': 0.1, 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100000}
Lowest MSLE:  0.7075395757140238


Ahora que hemos encontrado nuestros hiperparámetros óptimos, vamos a entrenar el modelo de nuevo

In [17]:
xgb_model = xgb.XGBRegressor(objective="count:poisson", 
max_depth=3, 
n_estimators=100000, 
learning_rate=0.1,
colsample_bytree=0.1, 
random_state=42  ) 

xgb_model.fit(X_train, y_train)

y_pred_xgb = xgb_model.predict(X_test)

print('XGBoost RMSLE :', np.sqrt(msle(y_test, y_pred_xgb)))
print('XGBoost mae :', mae(y_test, y_pred_xgb))

XGBoost RMSLE : 0.16931786935042142
XGBoost mae : 81.52235819498698


# Evaluación del modelo

A continuación, calcularemos el porcentaje promedio de desviación entre la predicción y el valor real

In [21]:
dataframe = pd.DataFrame()
dataframe['Num_viajes'] = y_test
dataframe['Num_viajes_p'] = y_pred_xgb
dataframe['dif'] = abs(dataframe['Num_viajes'] - dataframe['Num_viajes_p'])
import math
dataframe['Num_viajes_p'] =  dataframe['Num_viajes_p'].apply(lambda x: math.ceil(x))
dataframe["Dif_perc"] =  dataframe["dif"] / dataframe["Num_viajes"] *100
dataframe["Dif_perc"].mean()


13.067378986392685

Vemos que tenemos una desviación media de 13%, es decir, en promedio esta será la diferencia entre los viajes que se predicen y los que ocurren en realidad.

Este es nuestro mejor modelo hasta el momento, representando una mejora del 2% con respecto al modelo Random Forest

# Validación cruzada k-fold usando XGBoost 

Para construir modelos más robustos con XGBoost, deberíamos realizar siempre una validación cruzada k-fold.

De este modo, nos aseguramos de que el conjunto de datos de entrenamiento original se utiliza tanto para el entrenamiento como para la validación.

Además, cada entrada se utiliza para la validación sólo una vez.

XGBoost soporta la validación cruzada k-fold utilizando el método cv().

En este método, especificaremos varios parámetros que son los siguientes

* nfolds - Este parámetro especifica el número de conjuntos de validación cruzada que queremos construir.

* num_boost_round - Denota el número de árboles que construimos.

* metrics - Es la métrica de evaluación del rendimiento que se va a considerar durante la CV.

* as_pandas - Se utiliza para devolver los resultados en un DataFrame de pandas.

* early_stopping_rounds - Este parámetro detiene el entrenamiento del modelo antes de tiempo si la métrica de retención no mejora durante un número determinado de rondas.

* seed - Este parámetro se utiliza para la reproducibilidad de los resultados.

Podemos utilizar estos parámetros para construir un modelo de validación cruzada k-fold llamando al método CV() de XGBoost.

In [23]:
# Ahora, convertiremos el conjunto de datos en una estructura de datos optimizada llamada 
# Dmatrix que XGBoost admite y le proporciona aclamadas ganancias de rendimiento y eficiencia.
data_dmatrix = xgb.DMatrix(data=np.asarray(X),label=np.asarray(y))
data_dmatrix

In [24]:
from xgboost import cv

params = {"objective":"reg:squarederror",
                'max_depth': 5}

xgb_cv = cv(dtrain=data_dmatrix, params=params, nfold=5,
                    num_boost_round=50, early_stopping_rounds=10, as_pandas=True, seed=123)

xgb_cv contiene las métricas de precisión de entrenamiento y de prueba para cada ronda de boosting.

Veamos una vista previa de xgb_cv.

In [25]:
xgb_cv.head()


,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,805.965293,8.771106,813.380924,40.731484
1,645.116409,5.503807,661.165509,34.882705
2,538.979545,2.696400,565.811312,34.510692
3,467.521171,1.377258,503.126413,30.905573
4,418.636239,2.964750,460.651226,24.436093


# Feature importance con XGBoost

XGBoost proporciona una forma de examinar la importancia de cada característica en el conjunto de datos original dentro del modelo. Se trata de contar el número de veces que cada característica se divide en todos los árboles de boosting del modelo. A continuación, visualizamos el resultado en forma de gráfico de barras, con las características ordenadas según el número de veces que aparecen.

XGBoost tiene una función plot_importance() que nos ayuda a realizar esta tarea.

Así podemos visualizar la característica que ha recibido la mayor puntuación de importancia entre todas las características.

Por lo tanto, XGBoost nos proporciona una manera de hacer la selección de características.

Procederemos de la siguiente manera:-

In [28]:
import plotly.express as px
importance = pd.DataFrame()

importance["y"] = xgb_model.feature_importances_
importance["x"] = ['Num_escuelas',
 'Num_hospitales',
 'Num_iglesias',
 'Num_zonas_tipicas',
  "Hora_llegada_1",                                          
  "Hora_llegada_2",
  "Hora_llegada_3",                                          
  "Hora_llegada_4",
  "Hora_llegada_5",                                          
  "Hora_llegada_6",
  "Hora_llegada_7",                                          
  "Hora_llegada_8",
  "Hora_llegada_9",                                          
  "Hora_llegada_10",
  "Hora_llegada_11",                                          
  "Hora_llegada_12",
  "Hora_llegada_13",                                          
  "Hora_llegada_14",
  "Hora_llegada_15",                                          
  "Hora_llegada_16",                                         
  "Hora_llegada_17",                                          
  "Hora_llegada_18",
  "Hora_llegada_19",                                          
  "Hora_llegada_20",
  "Hora_llegada_21",                                          
  "Hora_llegada_22",
  "Hora_llegada_23" ]





px.pie(importance[0:4], names= "x", values = "y", title = "Importancia de cada variable determinada por el modelo XGBoost")

